In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV
import os
import glob

##### ***Data Generation***

In [130]:
trn_path='/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/'
val_path='/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_val_fea/'

In [131]:
def get_list_of_files(path):
    
    trn_files = glob.glob(os.path.join(path,"*_FEA.npy"))
    trn_labs= glob.glob(os.path.join(path,"*_FEA_LAB.npy"))
    return sorted(trn_files),sorted(trn_labs)

trn_files,trn_labs=get_list_of_files(trn_path)
val_files,val_labs=get_list_of_files(val_path)

observation

In [133]:
print(trn_files[0])
print(trn_labs[0])
print(val_files[0])
print(val_labs[0])
print(np.load(trn_files[0]).shape)
print(np.load(trn_labs[0]).shape)
print(np.load(trn_files[0][:-8]+'_XYZIR.npy').shape)##coodinate file

/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/JAX_004_FEA.npy
/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/JAX_004_FEA_LAB.npy
/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_val_fea/JAX_260_FEA.npy
/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_val_fea/JAX_260_FEA_LAB.npy
(155648, 128)
(155648, 1)
(155648, 5)


In [7]:
print(val_files[0][:-8])

/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_val_fea/JAX_260


In [13]:
print(np.load(trn_files[0]))

[[0.83554584 0.8009994  0.809932   ... 0.33603504 0.         0.4673449 ]
 [0.         0.08614694 0.         ... 0.         0.         0.        ]
 [0.56692135 1.211102   0.62345016 ... 0.76880795 0.         0.78496736]
 ...
 [0.22486682 1.0475065  0.20512994 ... 0.48153156 0.         0.35670096]
 [0.         0.         0.         ... 0.         0.6968867  0.90659523]
 [1.2816851  1.7837983  1.439342   ... 1.0023284  0.         1.3156325 ]]


In [13]:
print(np.unique(np.load(trn_labs[0])))

[0 2 5 6 9]


In [134]:
from tqdm import tqdm

for i in tqdm(range(len(trn_files))):
    temp_trndata=np.load(trn_files[i])
    temp_trnlab=np.load(trn_labs[i])
    if i==0:
        X_trn=temp_trndata
        y_trn=temp_trnlab
    else:
        X_trn=np.concatenate((X_trn,temp_trndata),axis=0)
        y_trn=np.concatenate((y_trn,temp_trnlab),axis=0)

print(X_trn.shape)
print(y_trn.shape)

100%|██████████| 4/4 [00:02<00:00,  1.91it/s]

(622592, 128)
(622592, 1)


In [135]:
y_trn=np.squeeze(y_trn)

###### las 2 sequence

In [136]:
y_trn_seq=y_trn
y_trn_seq[y_trn_seq==0]=255
y_trn_seq[y_trn_seq==2]=0
y_trn_seq[y_trn_seq==5]=1
y_trn_seq[y_trn_seq==6]=2
y_trn_seq[y_trn_seq==9]=3
y_trn_seq[y_trn_seq==17]=4

###### clean the sample that $\quad \text{y_{trn_seq}}==255$

In [137]:
trn_idx=y_trn_seq<255

In [138]:
X_trn_ult=X_trn[trn_idx]
y_trn_ult=y_trn_seq[trn_idx]

In [139]:
print(X_trn_ult.shape,y_trn_ult.shape)

(587746, 128) (587746,)


In [140]:
np.unique(y_trn_ult)

array([0, 1, 2, 3, 4], dtype=uint8)

random choosing samples to trn shorter time

In [141]:
idx=np.random.choice(X_trn_ult.shape[0],10000,replace=False)

X_trn_new=X_trn_ult[idx]
y_trn_new=y_trn_ult[idx]

print(X_trn_new.shape,y_trn_new.shape)

(10000, 128) (10000,)


###### sequence 2 las

In [142]:
LAB_MAP={0:2,1:5,2:6,3:9,4:17}

##### ***SVM***

In [143]:
from sklearn.svm import SVC

Gridsearch CV

In [ ]:
grid_SVM=GridSearchCV(SVC(decision_function_shape='ovo'), param_grid={'kernel':['linear'],
                        'C': [1]},cv=5)
grid_SVM.fit(X_trn,y_trn)
print("Best C is %f, Best gamma is %f" %(grid_SVM.best_params_['C'],grid_SVM.best_params_['gamma']))

In [ ]:
for mu_SVM,std_SVM,param_SVM in zip(grid_SVM.cv_results_['mean_test_score'],
                                   grid_SVM.cv_results_['std_test_score'],
                                   grid_SVM.cv_results_['params']):
    print('Mean test error: %.3f(+/-%.3f) with %r'%(mu_SVM,std_SVM,param_SVM))

Train & Save

In [144]:
from sklearn.externals import joblib

def trn_SVM(X_trn,y_trn,C):
    SVM_model=SVC(C=C,kernel='linear',decision_function_shape='ovo')
    SVM_model.fit(X_trn,y_trn)
    joblib.dump(SVM_model,'/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/SVM_model.m')
    return SVM_model

In [145]:
import time

time_start=time.time()
SVM_model=trn_SVM(X_trn_new,y_trn_new,1)
time_end=time.time()
print('training SVM model cost',time_end-time_start,'s')

training SVM model cost 2.361832857131958 s


Predict

In [146]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_SVM=[]
    y_pred=SVM_model.predict(X_val)
    y_pre_val_SVM=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(val_files[i][:-8]+'_SVM_LAB.npy',y_pre_val_SVM)
    print('OA:',np.mean(y_pre_val_SVM==y_val[:,0]))
time_end=time.time()
print('SVM model predicting cost',time_end-time_start,'s')

OA: 0.08571624755859375
OA: 0.05738525390625
SVM model predicting cost 77.61838841438293 s


In [147]:
np.unique(y_pre_val_SVM)

array([ 2,  5,  6,  9, 17])

Reload

In [67]:
SVM_model=joblib.load('/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/SVM_model.m')

In [68]:
y_pred=SVM_model.predict(X_val)
y_pre_val_SVM2=np.array([LAB_MAP[x] for x in y_pred])

In [69]:
print('OA:',np.mean(y_pre_val_SVM2==y_val[:,0]))

OA: 0.90489501953125


In [70]:
print(X_val.shape,y_val.shape)

(163840, 128) (163840, 1)


##### ***RF***

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
grid_RF=GridSearchCV(RandomForestClassifier(), 
                     param_grid={'n_estimators': [10,20,30,40,50,60,70,80,90,100]},cv=5)
grid_RF.fit(X_trn,y_trn)
print("Best number of estimators is %f" %(grid_RF.best_params_['n_estimators']))

In [72]:
def trn_RF(X_trn,y_trn,n_esti):
    RF_model=RandomForestClassifier(oob_score=True,n_estimators=n_esti,random_state=None,max_features='sqrt')#sqrt(c)
    RF_model.fit(X_trn,y_trn)
    joblib.dump(RF_model,'/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/RF_model.m')
    return RF_model

In [73]:
import time

time_start=time.time()
RF_model=trn_RF(X_trn_new,y_trn_new,100)
time_end=time.time()
print('training RF model cost',time_end-time_start,'s')

training RF model cost 7.401139259338379 s


Predict

In [74]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_RF=[]
    y_pred=RF_model.predict(X_val)
    y_pre_val_RF=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_RF==y_val))
time_end=time.time()
print('RF model predicting cost',time_end-time_start,'s')

OA: 0.2556835612631403
OA: 0.3753878366947174
RF model predicting cost 195.84533166885376 s


In [75]:
np.unique(y_pre_val_RF)

array([ 2,  5,  6,  9, 17])

##### ***RoF***

In [76]:
from rotation_forest import RotationTreeClassifier, RotationForestClassifier

In [ ]:
grid_RoF=GridSearchCV(RotationForestClassifier(), 
                     param_grid={'n_estimators': [10,20,30,40,50,60,70,80,90,100]},cv=5)
grid_RoF.fit(X_trn,y_trn)
print("Best number of estimators is %f" %(grid_RF.best_params_['n_estimators']))

In [77]:
def trn_RoF(X_trn,y_trn):
    sel_feat=int(np.sqrt(X_trn.shape[1]))
    RoF_model=RotationForestClassifier(n_estimators=20,random_state=None,max_features=sel_feat)#sqrt(c)
    RoF_model.fit(X_trn,y_trn)
    joblib.dump(RoF_model,'/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/RoF_model.m')
    return RoF_model

In [78]:
import time

time_start=time.time()
RoF_model=trn_RoF(X_trn_new,y_trn_new)
time_end=time.time()
print('training RoF model cost',time_end-time_start,'s')

training RoF model cost 33.654308557510376 s


In [79]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_RoF=[]
    y_pred=RoF_model.predict(X_val)
    y_pre_val_RoF=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_RoF==y_val))
time_end=time.time()
print('RoF model predicting cost',time_end-time_start,'s')

OA: 0.25628676800988615
OA: 0.3749700081720948
RoF model predicting cost 192.50943231582642 s


##### ***AB***

In [80]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
grid_AB=GridSearchCV(AdaBoostClassifier(), 
                     param_grid={'n_estimators': [10,20,30,40,50,60,70,80,90,100]},cv=5)
grid_AB.fit(X_trn,y_trn)
print("Best number of estimators is %f" %(grid_AB.best_params_['n_estimators']))

In [81]:
def trn_AB(X_trn,y_trn):
    AB_model=AdaBoostClassifier(n_estimators=100,random_state=None)#sqrt(c)
    AB_model.fit(X_trn,y_trn)
    joblib.dump(AB_model,'/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/AB_model.m')
    return AB_model

In [82]:
import time

time_start=time.time()
AB_model=trn_AB(X_trn_new,y_trn_new)
time_end=time.time()
print('training AB model cost',time_end-time_start,'s')

training AB model cost 15.588809490203857 s


In [84]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_AB=[]
    y_pred=AB_model.predict(X_val)
    y_pre_val_AB=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_AB==y_val))
time_end=time.time()
print('AB model predicting cost',time_end-time_start,'s')

OA: 0.2869176013045944
OA: 0.3730746987834573
AB model predicting cost 207.84754490852356 s


In [85]:
np.unique(y_val)

array([ 0,  2,  5,  6,  9, 17], dtype=uint8)

In [86]:
np.unique(y_pre_val_AB)

array([2, 5, 6])

##### ***GB***

In [87]:
from sklearn.ensemble import GradientBoostingClassifier

In [46]:
grid_GB=GridSearchCV(GradientBoostingClassifier(), 
                     param_grid={'n_estimators': [10,20,30,40,50,60,70,80,90,100],'learning_rate':[0.001,0.01,0.1]},cv=5)
grid_GB.fit(X_trn,y_trn)
print("Best number of estimators is %f" %(grid_GB.best_params_['n_estimators']))

KeyboardInterrupt: 

In [88]:
def trn_GB(X_trn,y_trn):
    GB_model=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,random_state=None)#sqrt(c)
    GB_model.fit(X_trn,y_trn)
    joblib.dump(GB_model,'/data/DW/DataFusion/dfc2019-master/track4/log_data_folder/model_fusion/test_trn_fea/GB_model.m')
    return GB_model

In [89]:
import time

time_start=time.time()
GB_model=trn_GB(X_trn_new,y_trn_new)
time_end=time.time()
print('training GB model cost',time_end-time_start,'s')

training GB model cost 85.37349510192871 s


In [90]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_GB=[]
    y_pred=GB_model.predict(X_val)
    y_pre_val_GB=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_GB==y_val))
time_end=time.time()
print('GB model predicting cost',time_end-time_start,'s')

OA: 0.27783634938532487
OA: 0.3759371126443148
GB model predicting cost 198.82611322402954 s


##### ***LGB***

In [91]:
import lightgbm as lgb

In [106]:
train_data=lgb.Dataset(X_trn_new,label=y_trn_new)
params={
    'learning_rate':0.01,
    'max_depth':1,
    'application':'multiclass',
    'num_class':5,
    'verbose':-1}

In [107]:
import time

time_start=time.time()
LGB_model=lgb.train(params,train_data)
time_end=time.time()
print('training LGB model cost',time_end-time_start,'s')

training LGB model cost 232.12101793289185 s


In [114]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_LGB=[]
    y_pred=LGB_model.predict(X_val)#one-hot
    y_pred_seq=np.argmax(y_pred,axis=1)
    y_pre_val_LGB=np.array([LAB_MAP[x] for x in y_pred_seq])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_LGB==y_val))
time_end=time.time()
print('LGB model predicting cost',time_end-time_start,'s')

OA: 0.28628260194091126
OA: 0.38213308673352003
LGB model predicting cost 189.33775854110718 s


##### ***XGB***

In [115]:
import xgboost as xgb

In [117]:
xgb_traindata = xgb.DMatrix(X_trn_new, label=y_trn_new) 

In [123]:
##https://blog.csdn.net/kwame211/article/details/81098025 
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 5,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 3,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4                   # cpu 线程数
}

In [126]:
import time

time_start=time.time()
num_round = 10
XGB_model = xgb.train(params, xgb_traindata, num_round)
time_end=time.time()
print('training XGB model cost',time_end-time_start,'s')

training XGB model cost 0.8037967681884766 s


In [129]:
time_start=time.time()
for i in range(len(val_files)):
    X_val=np.load(val_files[i])
    y_val=np.load(val_labs[i])
    y_pre_val_XGB=[]
    y_pred=XGB_model.predict(xgb.DMatrix(X_val))#one-hot
    print(np.unique(y_pred))
    y_pre_val_XGB=np.array([LAB_MAP[x] for x in y_pred])
    #np.save(os.path.join(path,"_RF_LAB.npy"),y_pre_val_RF)
    print('OA:',np.mean(y_pre_val_XGB==y_val))
time_end=time.time()
print('XGB model predicting cost',time_end-time_start,'s')

[0. 1. 2. 3. 4.]
OA: 0.21600613999180496
[0. 1. 2. 3. 4.]
OA: 0.3756059022992849
XGB model predicting cost 179.4974410533905 s


##### ***ANN***